# Third Notebook: Apply ORIs

Looks up & applies ORI codes from Crosswalk file to DMFS Cops & Civilians. Also records any manual matches or manual corrections that are made.

# 0 | Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter # Unique value ID

from thefuzz import fuzz # Fuzzy Search
import math
import os

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1 | Loads & Preprocesses Dataset

### 1.1 | Loads DFs

In [4]:
# Inits Directories
INPUT_DIR = "res/2.0 - DM-FS with Manual Corrections/"

In [5]:
# Loads DFs
df_civ = pd.read_csv(f"{INPUT_DIR}DM-FS with Manual Corrections.csv")
df_ori = pd.read_csv(f"{INPUT_DIR}ORI_Crosswalk.csv", low_memory = False)
df_cops = pd.read_csv(f"{INPUT_DIR}DM-FS - Officers before ORIs.csv")

### 1.2 | Preprocessing

In [6]:
# Fixes annoying spaces in df_ori column names
df_ori.columns = [x.strip() for x in list(df_ori.columns)]
df_ori.rename(columns={"NAME": "LEA_NAME"}, inplace=True) # Renames Name to LEA Name
df_ori.columns

Index(['FSTATE', 'FCOUNTY', 'FPLACE', 'FIPS_ST', 'FIPS_COUNTY', 'FIPS', 'ORI9',
       'ORI7', 'LEA_NAME', 'UA', 'STATENAME', 'COUNTYNAME', 'UANAME', 'PARTOF',
       'AGCYTYPE', 'SUBTYPE1', 'SUBTYPE2', 'GOVID', 'LG_NAME',
       'address_corrected', 'ADDRESS_NAME', 'ADDRESS_STR1', 'ADDRESS_STR2',
       'ADDRESS_CITY', 'ADDRESS_STATE', 'ADDRESS_ZIP', 'REPORT_FLAG',
       'CSLLEA08_ID', 'LEMAS_ID', 'U_STATENO', 'U_CNTY', 'U_POPGRP', 'U_TPOP',
       'LG_POPULATION', 'CSLLEA_SUB', 'COMMENT', 'INTPTLAT', 'INTPTLONG',
       'CONGDIST1', 'CONGDIST2_18', 'DISTNAME', 'SOURCE_CSLLEA2008',
       'SOURCE_UCR2010', 'SOURCE_UCR2011', 'SOURCE_UCR2012', 'SOURCE_NCIC2012',
       'SOURCE_VENDOR'],
      dtype='object')

In [7]:
# There's a ton of empty rows on the Crosswalk file, so let's only get the rows that have a value
df_ori = df_ori[df_ori["FSTATE"].notna()]
df_ori = df_ori.reset_index(drop = True)

In [8]:
# There's one agency in df_civ that wasn't seperated into LEA 1 and LEA 2 because a ';' was used instead of a ','. Let's fix that.
for bad_agency_index in list(df_civ[df_civ["agency_responsible_1"].str.contains(';', na=False)].index):
    problematic_value = df_civ.loc[bad_agency_index, "agency_responsible_1"]
    print(f"Multiple LEAs detected in df_civ at index {bad_agency_index} with value [{problematic_value}]. This has been corrected.")

    for i, ind_agency in enumerate(problematic_value.split(';')):
        df_civ.loc[bad_agency_index, f"agency_responsible_{i+1}"] = ind_agency.strip()

Multiple LEAs detected in df_civ at index 6614 with value [Weber County Sheriff's Office; Morgan County Sheriff's Office]. This has been corrected.


In [9]:
# DF_CIV has 6 columns for listing LEAs responsible for civillians deaths: one column per LEA
# However, Column #6 is never used: only up to 5 LEAs are responsible for a civillian's death
# Thus, we'll drop column #6

df_civ.drop(columns=['agency_responsible_6'], inplace=True)

In [10]:
# The databases list the "City of New York Police" as "New York Police". 
# This is confusing, because it may be confused with "New York State Police"
# Thus, to avoid ambiguity, New York Police will be renamed to City of New York Police in all the datasets

# Fixes it in Civillians
df_civ.loc[df_civ['agency_responsible_1'] == "New York Police Department", 'agency_responsible_1'] = "City of New York Police Department"

# Fixes it in Officers
df_cops.loc[df_cops['LEA_NAME'] == "New York Police Department", 'LEA_NAME'] = "City of New York Police Department"
df_cops.loc[df_cops['LEA_NAME'] == "New York Police Department, 46th Precinct", 'LEA_NAME'] = "City of New York Police Department"

# Fixes it in Crosswalk
df_ori.loc[df_ori['LEA_NAME'].str.startswith("NEW YORK CITY", na=False), 'LEA_NAME'] = df_ori['LEA_NAME'].str.replace(r'^NEW YORK CITY', 'CITY OF NEW YORK', regex=True)

# 2 | NLP

### 2.1 | Aux Functions

In [11]:
def remove_parentheses(text):

    if not text:
        return text

    if pd.isna(text):  
        return text
    
    result = []
    inside_parentheses = False
    for char in text:
        if char == '(':
            inside_parentheses = True
        elif char == ')':
            inside_parentheses = False
        elif not inside_parentheses:
            result.append(char)
    return ''.join(result).strip()

In [12]:
# Creates a dummy  name column. An abbreviated version of the victim's name we can use for comparison

def clean_LEA_name(df, col):

    # Creates a new column that's a carbon copy of the LEA name, and modifies that from hereon out
    df[f'{col}_CLEANED'] = df[col] # Old Col Name
    col = f'{col}_CLEANED'
    df[col] = df[col].fillna('') # Fills any NAs with empty string values (mainly for df_civ for agency_2, agency_3, etc.)
    df[col] = df[col].str.lower() # Makes lower

    # Fixes weird punctuation
    df[col] = df[col].str.replace("'", "")# Removes '
    df[col] = df[col].str.replace("\"", "")# Removes "
    df[col] = df[col].str.replace(".", "")# Removes .
    df[col] = df[col].str.replace("’", "")# Removes weird ’
    df[col] = df[col].str.replace("-", " ")# Removes -

    # Removes ',' and 'in' because that indicates a specific detachment of state police which likely doesn't have a unique ORI
    df[col] = df[col].str.split(',').str[0]
    df[col] = df[col].str.split(' in ').str[0]

    # Removes 'department' and 'office' b/c theyr'e sometimes used interchangeably between databases
    df[col] = df[col].str.replace("department", "")
    df[col] = df[col].str.replace("deparment", "")
    df[col] = df[col].str.replace("departmen", "")
    df[col] = df[col].str.replace(" dept", "")
    df[col] = df[col].str.replace(" office", "")

    # Removes the (parentheses) that WP adds to some LEAs as notes
    df[col] = df[col].apply(lambda x: pd.Series(remove_parentheses(x))) # Removes middle name(s)
        
    # Misc
    df[col] = df[col].str.replace("sheriffs", "sheriff")
    df[col] = df[col].str.replace("constables", "constable")
    df[col] = df[col].str.replace("bureau of", "")
    df[col] = df[col].str.replace("division of", "")
    df[col] = df[col].str.replace("borough", "")
#    df[col] = df[col].str.replace("housing authority", "") # housing authorities are usually part of some parent police department
    df[col] = df[col].str.replace("public safety", "police") # There is usually a disticntion "police" and "public safety", but they mean same for ORI Matching
    df[col] = df[col].str.replace("tech university", "tech") # Makes it more likely that, if a tech-uni LEA forgot to put uni in its name, it'll match (e.g., "Texas Tech Police" vs "Texas Tech University Police"
    df[col] = df[col].str.replace("metropolitan", "")
    df[col] = df[col].str.replace("of", "")
    df[col] = df[col].str.replace("county", "")
    df[col] = df[col].str.replace(" co ", " ") # 'co' is an abbreiated form of county

    # Spell out abbreviations
    df[col] = df[col].str.replace(" pd", " police")
    df[col] = df[col].str.replace(" twp ", " township ")
    df[col] = df[col].str.replace(" pct ", " precinct ")
#    df[col] = df[col].str.replace(" hp ", " highway patrol ")
    df[col] = df[col].str.replace(" independent school district", " isd") # Don't use the full "independent school district" abbreviation b/c otherwise there'll be a ton of false positives in the fuzzy search
    
    # Removes whitespace
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace("  ", " ")
    df[col] = df[col].str.replace(" ", "_")

### 2.2 | Applies NLP to LEA Names

In [13]:
# Cops
clean_LEA_name(df_cops, "LEA_NAME")
df_cops.loc[ : , ["LEA_NAME", "LEA_NAME_CLEANED"]].head(10)

,LEA_NAME,LEA_NAME_CLEANED
0,Johnson County Sheriff’s Office,johnson_sheriff
1,Hayward Police Department,hayward_police
2,San Jose Police Department,san_jose_police
3,"Colorado State Patrol, Golden",colorado_state_patrol
4,Okaloosa County Sheriff’s Office,okaloosa_sheriff
5,Coeur d’Alene Police Department,coeur_dalene_police
6,McHenry County Sheriff's Office,mchenry_sheriff
7,"Kentucky State Police, Mayfield Post",kentucky_state_police
8,U.S. Marshals Service of Baton Rouge,us_marshals_service_baton_rouge
9,Louisiana State Police in Lake Charles,louisiana_state_police


In [14]:
# Civs
for col_name in [f"agency_responsible_{x}" for x in range(1,6)]:
    clean_LEA_name(df_civ, col_name)

In [15]:
clean_LEA_name(df_ori, "LEA_NAME")
df_ori.loc[ : , ["LEA_NAME", "LEA_NAME_CLEANED"]].head(10)

,LEA_NAME,LEA_NAME_CLEANED
0,AUTAUGAVILLE POLICE DEPARTMENT ...,autaugaville_police
1,PRATTVILLE POLICE DEPARTMENT ...,prattville_police
2,PRATTVILLE FIRE DEPT ARSON INVESTIGATION BRANC...,prattville_fire_arson_investigation_branch
3,AUTAUGA COUNTY SHERIFF'S OFFICE ...,autauga_sheriff
4,FAULKNER STATE COMMUNITY COLLEGE POLICE DEPT ...,faulkner_state_community_college_police
5,BAY MINETTE POLICE DEPARTMENT ...,bay_minette_police
6,DAPHNE POLICE DEPARTMENT ...,daphne_police
7,ELBERTA POLICE DEPARTMENT ...,elberta_police
8,FAIRHOPE POLICE DEPARTMENT ...,fairhope_police
9,FOLEY POLICE DEPARTMENT ...,foley_police


### 2.3 | Processes State Names

In [16]:
# df_civs only has state abbreviations, but the crosswalk requires full state names for linkage
# Thus, let's turn the abbreviations into state names

state_abbr_to_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia'
}

# Creates a 'state_of_death' column
df_civ.rename(columns={"state_of_death": "state_of_death_abr"}, inplace=True)
df_civ["state_of_death"] = df_civ["state_of_death_abr"].map(state_abbr_to_name)

In [17]:
# Rearrange cols to place 'state_of_death' before 'state_of_death_abr'
cols = list(df_civ.columns)  # Get the current columns
abr_index = cols.index("state_of_death_abr")  # Correctly find the index of 'state_of_death_abr'
new_order = cols[:abr_index] + ["state_of_death"] + cols[abr_index:]  # Create new column order
df_civ = df_civ[new_order].iloc[:, :-1] # Drops last column b/c it's a duplicate of state_of_death

df_civ.head(1)

,date,victim_name,victim_age,victim_gender,victim_race,city_of_death,state_of_death,state_of_death_abr,agency_responsible_1,agency_responsible_2,agency_responsible_3,agency_responsible_4,agency_responsible_5,included_in_fe,included_in_mpv,included_in_wp,date_fe,date_mpv,date_wp,city_fe,city_mpv,city_wp,gender_fe,gender_mpv,gender_wp,age_fe,age_mpv,age_wp,race_fe,race_mpv,race_wp,agency_responsible_fe,agency_responsible_mpv,agency_responsible_wp,date_discrepancy_days,URL_FE,URL_MPV,URL_MANUAL_CORRECTION,agency_responsible_1_CLEANED,agency_responsible_2_CLEANED,agency_responsible_3_CLEANED,agency_responsible_4_CLEANED,agency_responsible_5_CLEANED
0,2020-12-31 00:00:00,David Randall Shephard,39,Male,White,Beaumont,Texas,TX,Beaumont Police Department,NaN,NaN,NaN,NaN,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,https://www.12newsnow.com/article/news/local/s...,https://www.usnews.com/news/best-states/texas/...,NaN,beaumont_police,,,,


In [18]:
# Let's make sure states are consistent within the 3 dataframes
df_cops["LEA_STATE"] = df_cops["LEA_STATE"].str.strip().str.title()
df_ori["STATENAME"] = df_ori["STATENAME"].str.strip().str.title()
df_civ["state_of_death"] = df_civ["state_of_death"].str.strip().str.title()

In [19]:
# Sanity check: do all datasets have the same states? Puerto Rico is OK

unique_cops = set(df_cops["LEA_STATE"].unique())
unique_ori = set(df_ori["STATENAME"].unique())
unique_civ = set(df_civ["state_of_death"].unique())

# Check for unique values in each list that do not appear in the others
unique_to_cops = unique_cops - unique_ori - unique_civ
unique_to_ori = unique_ori - unique_cops - unique_civ
unique_to_civ = unique_civ - unique_cops - unique_ori

# Print results
print("Unique States in DM-FS: Officers")
print(unique_to_cops)

print("Unique States in the Crosswalk")
print(unique_to_ori)

print("Unique States in DM-FS: Civillians")
print(unique_to_civ)

Unique States in DM-FS: Officers
{'Puerto Rico'}
Unique States in the Crosswalk
set()
Unique States in DM-FS: Civillians
set()


# 3 | Loop Auxilary Functions

### 3.1 | Match Methods: Manual Verification & Adding Match to DF

In [20]:
# Asks user to confirm match
# Loops until user types in Y or N

def manually_verify_match():
    while True:
        user_input = input("Confirm Match? Type 'Y' for Yes or 'N' for No: ").strip().upper()
        if user_input in ['Y', 'N']:
            return user_input
        else:
            print("Invalid input. Please enter 'Y' for Yes or 'N' for No.")

In [21]:
# Adds a new row to the match verification DF
def add_row_to_match_verification_df(db1_name, candidate_index, candidate_row, candidate_name, candidate_full_name, user_input, agency_responsible_num = -1):
    next_index = len(match_verification_df)
    match_verification_df.loc[next_index] = {"db_name" : db1_name, "agency_responsible_number" : agency_responsible_num, 
                                             "db_index" : i, "csslea_index" : candidate_index,
                                             "lea_name_db" : name, "lea_name_csllea" : candidate_name , 
                                            "lea_full_name_db" : full_name, "lea_full_name_csllea" : candidate_full_name,
                                             "lea_state_db" : state, "lea_state_csllea" : candidate_row["STATENAME"].strip().title(),
                                            "lea_ori_7" : candidate_row["ORI7"], "lea_ori_9" : candidate_row["ORI9"], "match_verified" : (user_input == "Y")}

### 3.2 | Fuzzy Match

In [22]:

def get_candidate_rows_via_fuzzzy_name_search(db, db1_name, agency_responsible_num = -1, threshold = 80):

    candidate_indices = []

    last_fuzzy_match_full_name, last_fuzzy_match_decision = None, None
    for candidate_index, candidate_row in db.iterrows():

        # Calculates fuzzy match
        candidate_name, candidate_full_name = candidate_row['LEA_NAME_CLEANED'], candidate_row['LEA_NAME'].strip().title()
        fuzzy_match = fuzz.ratio(name, candidate_name) > threshold

        if fuzzy_match:

            # Checks to see if it's already been matched. T His is because it's common for lEAs of the same name (e.g., New York) to have MANY ENTRIES
            verification_record = match_verification_df.query(f"db_name == '{db1_name}' & agency_responsible_number == {agency_responsible_num} & db_index == {i} & csslea_index == {candidate_index}")

            # If a match hasn't been verified, have user verify it
            if verification_record.empty:

                # Manually confirms fuzzy match
                print(f"{candidate_row['STATENAME']}: Fuzzy match for [{name}] and [{candidate_name}] ([{full_name}] and [{candidate_full_name})]?")
                if last_fuzzy_match_full_name and candidate_full_name == last_fuzzy_match_full_name: # If a repeat was detected (e.g., Philidalphia Police which has >100 entries)
                    print(f"--> Repeat entry detected. Using decision ['{user_input}'], the same as last time ")
                    user_input = last_fuzzy_match_decision
                else: # No duplicate
                    user_input = manually_verify_match()
                
                # Adds the user's decision to the dataframe (with ORI code)
                add_row_to_match_verification_df(db1_name, candidate_index, candidate_row, candidate_name, candidate_full_name, user_input, agency_responsible_num)
                last_fuzzy_match_full_name, last_fuzzy_match_decision = candidate_full_name, user_input # Stores the user's last decision
            
            # If a mach has been verified, re-use that same decision
            else:
                match_verified = verification_record["match_verified"].iloc[0]
                user_input = "Y" if match_verified else "N"
            
            # If match was verified, add it to our search results
            if user_input == 'Y':
                candidate_indices.append(candidate_index)
 
    return db.loc[candidate_indices]

### 3.3 | Initialize Match

Contains different matching proceure for 0 matches, 1 matches, and multiple matches

In [23]:
# Takes ORI Candidates (a pandas dataframe) and returns either None or a match

def initialize_ori_match_from_candidates(candidates, db1_name, agency_responsible_num = -1):
    # If there's 0 or 1 candidate
    if len(candidates) == 0:
        return None
    elif len(candidates) == 1:
        return candidates.iloc[0]

    # If there are multiple candidates...
    print(f"{name} has multiple candidates {len(candidates)}")

    # Select only the parent agencies
    candidates_parent_agency_only = candidates[ candidates["PARTOF"].str.contains("Parent agency", na=False) ]

    # If there's only 1 parent agency in the list, return that
    if len(candidates_parent_agency_only) == 1:
        return candidates_parent_agency_only.iloc[0]

    # **********************************************************************
    # Handles multiple parent agencies via user input
    # **********************************************************************
    
    # If there are multiple parents, then there are multiple very different agencies thrown in here.
    # Return "None" so we can manually review it.
    elif len(candidates_parent_agency_only) > 1:
            
        # Prints out all the candidates
        print(f"Uh oh! Multiple parent agencies! Please identify the ORI for [{name}] ([{full_name}])")
        for index, ind_match in candidates_parent_agency_only.iterrows():
            print(f"--> {index} | {ind_match['LEA_NAME'].strip().title()} | {ind_match['LEA_NAME_CLEANED']} | {ind_match['ORI7']} | {ind_match['ORI9']}")
        print("We will go one-by-one. Please select the match you want to use")

        # Go row-by-row, asking user which one they'd like to select as candidate
        index_to_use = -1
        for candidate_index, candidate_row in candidates_parent_agency_only.iterrows():

            # Checks to see if it's already been matched. T His is because it's common for lEAs of the same name (e.g., New York) to have MANY ENTRIES
            verification_record = match_verification_df.query(f"db_name == '{db1_name}' & agency_responsible_number == {agency_responsible_num} & db_index == {i} & csslea_index == {candidate_index}")
            
            if verification_record.empty:
                print(f"--> {candidate_index} | {candidate_row['LEA_NAME'].strip().title()} | {candidate_row['LEA_NAME_CLEANED']} | {candidate_row['ORI7']} | {candidate_row['ORI9']}")
                user_input = manually_verify_match()
                add_row_to_match_verification_df(db1_name, candidate_index, candidate_row, candidate_row['LEA_NAME_CLEANED'], candidate_row['LEA_NAME'].strip(), 
                                                 user_input, agency_responsible_num)
            else:
                match_verified = verification_record["match_verified"].iloc[0]
                user_input = "Y" if match_verified else "N"

            # Updates index_to_use wiht whatever user selects/has selected
            index_to_use = candidate_index if user_input == 'Y' else index_to_use
            
        print("Using index ",index_to_use)
        return None if index_to_use == -1 else candidates_parent_agency_only.loc[index_to_use]

    # **********************************************************************
    # Handles multiple child agencies
    # **********************************************************************

    # If there's only one child agency with a valid ORI7, return that.
    candidates_with_ori = candidates_parent_agency_only[ ~candidates_parent_agency_only["ORI7"].str.contains("-1", na=False) ]
    if len(candidates_with_ori) == 1:
        return candidates_with_ori.iloc[0]

    # If there's only one child agency with a valid ORI9, return that
    candidates_with_ori = candidates_parent_agency_only[ ~candidates_parent_agency_only["ORI9"].str.contains("-1", na=False) ]
    if len(candidates_with_ori) == 1:
        return candidates_with_ori.iloc[0]
        
    # If there's multiple children with no clear ORI or parent, just arbitrarily use the first one.
    return candidates.iloc[0]

### 3.4 | Records Manual Matches

In [24]:
# Records all manual ORI matches for transparency/replication
def record_manual_match(db_name, agency_responsible_num, og_lea, new_lea, new_lea_index, ori7, ori9, notes):
    new_row = {"db_name" : db_name, "agency_responsible_num" : agency_responsible_num, 
               "original_LEA_name" : og_lea, "LEA_that_original_was_mapped_onto_for_ORI_code" : new_lea, 
               "LEA_that_original_was_mapped_onto_csllea_index" : new_lea_index, "ORI7" : ori7, "ORI9" : ori9, "Notes" : notes}
    manual_corrections_df.loc[len(manual_corrections_df)] = new_row

### 3.5 | Add & Initilaize CSLLEA Cols for Police

In [25]:
def add_csllea_cols_to_df_cops(df):
    cols_to_add = ["ORI7", "ORI9", "LEA_TYPE", "LEA_SUBTYPE_1", "LEA_SUBTYPE_2", "COUNTY", "LOCAL_GOVERNMENT_NAME", "DISTRICT", "FIPS_STATE", "FIPS_COUNTY", "FCOUNTY", "FPLACE"]
    for ind_col in cols_to_add:
        df[ind_col] = None
                          

In [26]:
# Initializes a new row to add to the cops dataset
def initialize_match_to_add_for_dmfs_cops(ind_candidate):

    # Initializes match_to_add if there's a candidate
    if not ind_candidate.empty:
        match_to_add = {"ORI7" : ind_candidate['ORI7'].strip(), "ORI9" : ind_candidate['ORI9'].strip(), 
                        "LEA_TYPE" : ind_candidate['AGCYTYPE'].strip(), "LEA_SUBTYPE_1" : ind_candidate['SUBTYPE1'].strip(),
                        "LEA_SUBTYPE_2" : ind_candidate['SUBTYPE2'].strip(), "COUNTY" : ind_candidate['COUNTYNAME'].strip().title(),
                        "LOCAL_GOVERNMENT_NAME" : ind_candidate['LG_NAME'].strip().title(), "DISTRICT" : ind_candidate['DISTNAME'].strip().title(), 
                        "FIPS_STATE" : int(ind_candidate['FIPS_ST']), "FIPS_COUNTY" : int(ind_candidate['FIPS_COUNTY']), 
                        "FCOUNTY" : int(ind_candidate['FCOUNTY']), "FPLACE" : int(ind_candidate['FPLACE'])}

    # If there's no candidate, fill it with blank values
    else:
        match_to_add = {"ORI7" : -1, "ORI9" : -1, 
                        "LEA_TYPE" : "N/A", "LEA_SUBTYPE_1" : "N/A", "LEA_SUBTYPE_2" : "N/A", "COUNTY" : "N/A",
                        "LOCAL_GOVERNMENT_NAME" : "N/A", "DISTRICT" : "N/A", 
                        "FIPS_STATE" : -1, "FIPS_COUNTY" : -1, 
                        "FCOUNTY" : -1, "FPLACE" : -1 }
    return match_to_add

### 3.6 | Add & Initialize CSLLEA Cols for CIVILIANS

In [27]:
def add_csllea_cols_to_df_civs(df):
    cols_to_add = ["ORI7", "ORI9", "LEA_TYPE", "LEA_SUBTYPE_1", "LEA_SUBTYPE_2", "LOCAL_GOVERNMENT_NAME", "DISTRICT", "FIPS_STATE", "FIPS_COUNTY"]

    # Loop through the agency_responsible_1 to agency_responsible_5
    for x in range(1, 6):
        # Find the index of the current 'agency_responsible_x' column
        agency_col = f"agency_responsible_{x}"
        if agency_col in df.columns:
            insert_loc = df.columns.get_loc(agency_col) + 1  # Insert after the agency column

            for ind_col in cols_to_add:
                new_col_name = f"agency_responsible_{x}_{ind_col}"
                # Insert the new column right after the agency column
                df.insert(insert_loc, new_col_name, None)
                insert_loc += 1  # Move the insertion location for the next column
        else:
            print(f"Column {agency_col} not found in the DataFrame.")

In [28]:
def initialize_match_to_add_for_dmfs_civs(ind_candidate, agency_responsible_col):
    
    # Initializes match_to_add if there's a candidate
    if not ind_candidate.empty:
        match_to_add = {f"{agency_responsible_col}_ORI7" : ind_candidate['ORI7'].strip(), f"{agency_responsible_col}_ORI9" : ind_candidate['ORI9'].strip(), 
                        f"{agency_responsible_col}_LEA_TYPE" : ind_candidate['AGCYTYPE'].strip(), f"{agency_responsible_col}_LEA_SUBTYPE_1" : ind_candidate['SUBTYPE1'].strip(),
                        f"{agency_responsible_col}_LEA_SUBTYPE_2" : ind_candidate['SUBTYPE2'].strip(), 
                        f"{agency_responsible_col}_LOCAL_GOVERNMENT_NAME" : ind_candidate['LG_NAME'].strip().title(), 
                        f"{agency_responsible_col}_DISTRICT" : ind_candidate['DISTNAME'].strip().title(), 
                        f"{agency_responsible_col}_FIPS_STATE" : int(ind_candidate['FIPS_ST']), f"{agency_responsible_col}_FIPS_COUNTY" : int(ind_candidate['FIPS_COUNTY'])}

    # Otherwise, fill these values up with placeholders
    else:
        match_to_add = {f"{agency_responsible_col}_ORI7" : -1, f"{agency_responsible_col}_ORI9" : -1, 
                        f"{agency_responsible_col}_LEA_TYPE" : "N/A", f"{agency_responsible_col}_LEA_SUBTYPE_1" : "N/A",
                        f"{agency_responsible_col}_LEA_SUBTYPE_2" : "N/A", 
                        f"{agency_responsible_col}_LOCAL_GOVERNMENT_NAME" : "N/A", 
                        f"{agency_responsible_col}_DISTRICT" : "N/A", 
                        f"{agency_responsible_col}_FIPS_STATE" : -1, f"{agency_responsible_col}_FIPS_COUNTY" : -1}
    return match_to_add

# 4 | Add ORIs to DMFS: Cops

### 4.1 | Init Key Variables

In [29]:
# Adds CSLLEA data placeholders
add_csllea_cols_to_df_cops(df_cops)

USE_PRE_EXISTING_MATCH_DATAFRAME = True
MATCH_DIR = "res/3.0 - Manual ORI Matches/"

# Records any manual corrections / overrides regarding ORI code assignment
manual_corrections_df = pd.DataFrame(columns=["db_name", "agency_responsible_num", "original_LEA_name", "LEA_that_original_was_mapped_onto_for_ORI_code", "LEA_that_original_was_mapped_onto_csllea_index", "ORI7", "ORI9", "Notes"])

# Creates a new dataframe to store manual matches
if USE_PRE_EXISTING_MATCH_DATAFRAME:
    print("Using pre-existing manual match dataframe")
    match_verification_df = pd.read_csv(f"{MATCH_DIR}manual_ori_matches.csv")
else:
    print("Creating new manual match dataframe. You will need to manually confirm any match ambiguities in the for loop.")
    match_verification_df =  pd.DataFrame(columns=["db_name", "agency_responsible_number", #agency_responsible_number is only for df_civ
                                                   "db_index", "csslea_index", "lea_name_db", "lea_name_csllea", "lea_full_name_db", "lea_full_name_csllea", 
                                                   "lea_state_db", "lea_state_csllea",
                                                   "lea_ori_7", "lea_ori_9", "match_verified"])

match_verification_df.head(1)

Using pre-existing manual match dataframe


,db_name,agency_responsible_number,db_index,csslea_index,lea_name_db,lea_name_csllea,lea_full_name_db,lea_full_name_csllea,lea_state_db,lea_state_csllea,lea_ori_7,lea_ori_9,match_verified
0,dmfs_officers,-1,5,6466,coeur_dalene_police,couer_dalene_police,Coeur d’Alene Police Department,Couer D'Alene Police Department,Idaho,Idaho,ID02801,ID0280100,True


### 4.2 | Main Loop

Loops through each row of DMFS-Officers. Takes the LEA name of each row and pairs it with the equivlent name in CSLLEA. Finally, once a match is found, import key information from that match into DMFS (e.g., ORI codes, FIPS codes, etc.)

In [30]:
# DMFS-Officers
for i in range(0, len(df_cops) ) :
    print(f"Processing index {i} out of {len(df_cops)} ({round(i/len(df_cops) * 100, 2)}%)")
    lea_row = df_cops.iloc[i].copy()
    name, full_name, state = lea_row["LEA_NAME_CLEANED"], lea_row["LEA_NAME"], lea_row["LEA_STATE"]


    # *************************************
    # Checks for a match in other DBs
    # *************************************

    # Name & State Match
    ori_candidates = df_ori.query(f"LEA_NAME_CLEANED == '{name}' & STATENAME == '{state}'")# & city_of_death_abr == '{city}'")

    # If no match, then salavage it with a fuzzy search
    if ori_candidates.empty:
        print("Trying fuzzy match for ",name)
        ori_candidates = get_candidate_rows_via_fuzzzy_name_search (df_ori[ (df_ori['STATENAME'] == state) ], "dmfs_officers",)

    # Extracts a match from the candidates
    ind_candidate = initialize_ori_match_from_candidates(ori_candidates, "dmfs_officers")
    
    if ind_candidate is None:
        print(f"No match for {name}")
        continue
        
    match_to_add = initialize_match_to_add_for_dmfs_cops(ind_candidate)
    df_cops.loc[i, match_to_add.keys()] =  match_to_add.values()

Processing index 0 out of 232 (0.0%)
Processing index 1 out of 232 (0.43%)
Processing index 2 out of 232 (0.86%)
Processing index 3 out of 232 (1.29%)
colorado_state_patrol has multiple candidates 2
Processing index 4 out of 232 (1.72%)
Processing index 5 out of 232 (2.16%)
Trying fuzzy match for  coeur_dalene_police
Processing index 6 out of 232 (2.59%)
Processing index 7 out of 232 (3.02%)
kentucky_state_police has multiple candidates 2
Processing index 8 out of 232 (3.45%)
Trying fuzzy match for  us_marshals_service_baton_rouge
No match for us_marshals_service_baton_rouge
Processing index 9 out of 232 (3.88%)
louisiana_state_police has multiple candidates 14
Processing index 10 out of 232 (4.31%)
Trying fuzzy match for  housing_authority_new_orleans_police
No match for housing_authority_new_orleans_police
Processing index 11 out of 232 (4.74%)
new_orleans_police has multiple candidates 10
Processing index 12 out of 232 (5.17%)
shreveport_police has multiple candidates 149
Processing

### 4.3 | Apply Manual Corrections to Missed ORIs

In [31]:
# Contains dmfs_index : [csllea_index, notes] mapping
manual_corrections = {
    10 : [11692, "Housing Authority of New Orleans Police mapped onto New Orleans Police (generic) because former does not have ORI code"],
    127 : [2366, "Manually located Ranco Corvo PD"],
    181 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    189 : [16776, "North County Police Co-Op doesn't have an ORI code. However, it is located in Vinita Terrace, and there's a \"Vinita Terrace PD\" listed on the website of \"North County Police Co-Op\". Thus, we used the ORI of Vinita Terrace"],
    201 : [32807, "Mapped Texas Highway Patrol onto its headquarters rather than a satellite location."],
}

In [32]:
# Loops thorugh ORIs with missing data & corrects them

missed_LEAs = df_cops.query(f"ORI7.isnull()")
print(f"There are {df_cops.shape[0]} LEAs, of which {missed_LEAs.shape[0]} have a missing ORI ({ round(missed_LEAs.shape[0] / df_cops.shape[0] * 100, 2)}% of total).")
print(f"Applying {len(manual_corrections.keys())} manual corrections to the {missed_LEAs.shape[0]} agencies with missing ORIs, thereby remedying {round(len(manual_corrections.keys()) / missed_LEAs.shape[0] * 100, 2)}% of missing ORIs")

for index, ind_LEA in missed_LEAs.iterrows():
    
    # Applies manual correctioon, if it exists
    if index in manual_corrections.keys():
        ori_index = manual_corrections[index][0]
        match_to_add = initialize_match_to_add_for_dmfs_cops( df_ori.loc[ ori_index ] ) # Adds to DM-FS
        record_manual_match("dmfs_officers", "N/A", ind_LEA["LEA_NAME"], df_ori.loc[ori_index, "LEA_NAME"].strip().title(), 
                            ori_index, df_ori.loc[ori_index, "ORI7"], df_ori.loc[ori_index, "ORI9"], manual_corrections[index][1]) # Records manual match for replication

    # Else, just create a bunch of blank/NA cells to add
    else:
        match_to_add = initialize_match_to_add_for_dmfs_cops( pd.Series() )
        record_manual_match("dmfs_officers", "N/A", ind_LEA["LEA_NAME"], "No Match Found", -1, -1, -1, notes="")
        
    # Adds data to DM-FS
    df_cops.loc[index, match_to_add.keys()] =  match_to_add.values()

There are 232 LEAs, of which 21 have a missing ORI (9.05% of total).
Applying 5 manual corrections to the 21 agencies with missing ORIs, thereby remedying 23.81% of missing ORIs


### 4.4 |  Exports

In [33]:
# Exports Manual Match Dataframe, the one that records all our manual matches (if appliable)
if not USE_PRE_EXISTING_MATCH_DATAFRAME:
    print(f"Saving manual ORI matches to dir [{MATCH_DIR}] so that the manual matches can be replicated in a future run.")
    match_verification_df.to_csv(f"{MATCH_DIR}manual_ori_matches.csv", index = False)
else:
    print("Used pre-existing manual match dataframe. That dataframe was not modified in any way, hence, no saving is necessary.\nNo action is needed. Please proceed.")

Used pre-existing manual match dataframe. That dataframe was not modified in any way, hence, no saving is necessary.
No action is needed. Please proceed.


In [34]:
# Also exports any manual corrections we've made
print(f"Exporting manual corrections to [{MATCH_DIR}]")
manual_corrections_df.to_csv(f"{MATCH_DIR}manual_ori_corrections.csv", index = False)

Exporting manual corrections to [res/3.0 - Manual ORI Matches/]


In [35]:
# Exports DMFS-Officers
EXPORT_DIR = "res/3.1 - Penultimate Databases/"
if not os.path.exists(EXPORT_DIR):
    os.makedirs(EXPORT_DIR)
    
df_cops.drop(columns=["LEA_NAME_CLEANED"], inplace=True)
df_cops.to_csv(f"{EXPORT_DIR}DMFS Officers.csv", index=False)

# 5 | Add ORI to DMFS-Civilians

### 5.1 | Pre Loop Corrections

While applying manual corrections, we discovered some rows that had a mistake like an erroneous LEA attribution or a mistaken state of death. Our earlier steps didn't detect these issues because, in these cases, two/three databases actually AGREED on the erroneous bit of information. If all 3 databases, for example, claimed that John Doe was killed by Chicago Police when in actuality, he was killed by Illinois State Police, there was no reason to flag it because we assumed that, if all 3 databases agree, the information is generally reliable.

Unfortunately, there were a few rare cases where that assumption didn't pan out, and these are corrected below, before the ORIs are assigned in the main loop.

In [36]:
# Corrects an issue of LEA Attribution discovered while correcting these manual discrepancies.
# In other words, all 3 databases claim some Georgia sheriff's department fatally shot 2 civillians in Maine.
# This is wrong: it was Maine State Troopers and the local LEA. The below code fixes that. See https://fox23maine.com/news/local/report-state-police-justified-in-controversial-shooting-of-man-woman-in-vassalboro
df_civ.loc[4345, ["agency_responsible_1", "agency_responsible_2"]] = ["Maine State Police", "Vassalboro Police",]
df_civ.loc[4351, ["agency_responsible_1", "agency_responsible_2"]] = ["Maine State Police", "Vassalboro Police",]


# Other Mistakes
df_civ.loc[1291, "state_of_death"] = "Kansas" # Claims Pittsburg was in MO when, in actuality, it's in Kansas.
df_civ.loc[1291, "state_of_death_abr"] = "KS"
df_civ.loc[303, "state_of_death"] = "Florida" # Article confirms victim perished in Florida, not Tennessee.
df_civ.loc[303, "state_of_death_abr"] = "FL"
df_civ.loc[1171, "state_of_death"] = "Alabama" # Mobile police is located in Alabama, not Arkansas.
df_civ.loc[1171, "state_of_death_abr"] = "AL"
df_civ.loc[3080, "agency_responsible_1"] = "Corpus Christi Police Department" #DBs list just "Corpus Christi", which has a marshal, sheriff, etc. Article confirms it's the PD
df_civ.loc[2045, "agency_responsible_1"] = "Hertford County Sheriff's Office" # DBs originally said "Hertford County Chief Deputy Will Liverman" was agency responsible
df_civ.loc[5376, "agency_responsible_1"] = "Harris County Constable Precinct 6" # Changes entry from "Harris COunty Constable" to the specific precinct that fatally shot the man.


### 5.2 | Init Key Variables

In [37]:
# Adds ORI cols to dmfs-civ
add_csllea_cols_to_df_civs(df_civ)

### 5.3 | Main Loop

Similar to §4.2, the main loop goes row-by-row in DM-FS. Namely, it extracts (i) LEA name and (ii) state-of-death from DM-FS. It then pairs it with the equivlent LEA in the Crosswalk file (CSLLEA) to extract its LEA. 

If an LEA name cannot be found, the name is salvaged via a fuzzy search, and the user needs to confirm the fuzzy match is valid before proceeding. All fuzzy match decisions are recorded in the "3.0 - Manual ORI Matches" directory  by default, under 'manual_ori_matches.csv'.

If multiple LEA names are returned, then it asks the user to select the LEA responsible, and the user's decisions are recorded in the aforementioned CSV file. This was most common when there were two LEAs with extremely similar names but served different purposes, e.g., "Baltimore Police" vs. "Baltimore County Polcie"

In [38]:

# Goes one agency at a time: agency_responsible_1, agency_responsible_2 [...], agency_responsible 5
for agency_responsible_num in range(1,6):
    
    agency_responsible_col = f"agency_responsible_{agency_responsible_num}"
    print(f"\n{'*'*80}\nMATCHING AGENCIES IN [{agency_responsible_col}] WITH AN ORI\n{'*'*80}\n")

    # Goes one-by-one, pairing each agency in DMFS-Civs with its ORI in the Crosswalk file
    for i in range(0, len(df_civ) ) :
    
    
        lea_row = df_civ.iloc[i].copy()
        name, full_name, state = lea_row[f"{agency_responsible_col}_CLEANED"], lea_row[agency_responsible_col], lea_row["state_of_death"]

        # Checks for empty values. If so, it doesn't print out anything & just continues with the loop
        if name == "":
            df_civ.loc[i, agency_responsible_col] = "N/A"
            continue
        else:
            print(f"Processing index {i} out of {len(df_civ)} ({round(i/len(df_civ) * 100, 2)}%)")

        # *************************************
        # Checks for a match in Crosswalk file
        # *************************************

        # Name & State Match
        ori_candidates = df_ori.query(f"LEA_NAME_CLEANED == '{name}' & STATENAME == '{state}'")

        # If no match, then salavage it with a fuzzy search
        if ori_candidates.empty:
            print("Trying fuzzy match for ",name)
            ori_candidates = get_candidate_rows_via_fuzzzy_name_search (df_ori[ (df_ori['STATENAME'] == state) ], "dmfs_civilians", agency_responsible_num)
    
        # Extracts a match from the candidates
        ind_candidate = initialize_ori_match_from_candidates(ori_candidates, "dmfs_civilians", agency_responsible_num)
    
        if ind_candidate is None:
            print(f"No match for {name}")
            continue

        match_to_add = initialize_match_to_add_for_dmfs_civs(ind_candidate, agency_responsible_col)
        df_civ.loc[i, match_to_add.keys()] =  match_to_add.values()


********************************************************************************
MATCHING AGENCIES IN [agency_responsible_1] WITH AN ORI
********************************************************************************

Processing index 0 out of 7105 (0.0%)
Processing index 1 out of 7105 (0.01%)
miami_police has multiple candidates 2
Processing index 2 out of 7105 (0.03%)
pennsylvania_state_police has multiple candidates 28
Processing index 3 out of 7105 (0.04%)
Processing index 4 out of 7105 (0.06%)
minneapolis_police has multiple candidates 3
Processing index 5 out of 7105 (0.07%)
Processing index 6 out of 7105 (0.08%)
Processing index 7 out of 7105 (0.1%)
Processing index 8 out of 7105 (0.11%)
Trying fuzzy match for  us_marshals_service
No match for us_marshals_service
Processing index 9 out of 7105 (0.13%)
Processing index 10 out of 7105 (0.14%)
city_new_york_police has multiple candidates 2
Processing index 11 out of 7105 (0.15%)
Processing index 12 out of 7105 (0.17%)
Processing 

### 5.4 | OPTIONAL: Inspects data for manual corrections

When applying manual corrections, we needed to pair (i) the LEA without an ORI code in DM-FS (ii) the correct LEA in the Crosswalk file. Thus, we needed to identify two indices: the original index in DM-FS (of the LEA with a missing ORI) and the corrected index in the crosswalk file. These two functions helped us manually lookup these indices so we can add them to our dictionary in the next section.

This code is not essential unless the user wishes to initiate additional manual corrections. It can be safely skipped.

In [39]:
# Want to get an ORI via a CSLLEA index? 
# Type the LEA name you wish to query and copy its index
lea_name_to_query = "UMATILLA TRIBAL POLICE DEPARTMENT"
df_ori[df_ori["LEA_NAME"].str.strip().str.startswith(lea_name_to_query)]

,FSTATE,FCOUNTY,FPLACE,FIPS_ST,FIPS_COUNTY,FIPS,ORI9,ORI7,LEA_NAME,UA,STATENAME,COUNTYNAME,UANAME,PARTOF,AGCYTYPE,SUBTYPE1,SUBTYPE2,GOVID,LG_NAME,address_corrected,ADDRESS_NAME,ADDRESS_STR1,ADDRESS_STR2,ADDRESS_CITY,ADDRESS_STATE,ADDRESS_ZIP,REPORT_FLAG,CSLLEA08_ID,LEMAS_ID,U_STATENO,U_CNTY,U_POPGRP,U_TPOP,LG_POPULATION,CSLLEA_SUB,COMMENT,INTPTLAT,INTPTLONG,CONGDIST1,CONGDIST2_18,DISTNAME,SOURCE_CSLLEA2008,SOURCE_UCR2010,SOURCE_UCR2011,SOURCE_UCR2012,SOURCE_NCIC2012,SOURCE_VENDOR,LEA_NAME_CLEANED
25426,(41) Oregon,59.0,57150.0,41.0,59.0,41059.0,OR0301200,OR03012,UMATILLA TRIBAL POLICE DEPARTMENT ...,68428.0,Oregon,UMATILLA,"Pendleton, OR Urban Cluster ...",(0) Parent agency,(000) Local police department,(888) Not applicable,(888) Not applicable,387030001.0,CONFEDERATED TRIBES OF THE UMATILLA INDIAN RES...,UMATILLA TRIBAL POLICE DEPARTMENT,UMATILLA TRIBAL POLICE DEPARTMENT ...,73320 HIGHWAY 331,PO BOX 638,PENDLETON,OR,97801,(1) ORI reported 1+ offenses,12085690.0,-1.0,(36) Oregon,-2.0,7,0.0,999999999.0,(1) Include record to appoximate CSLLEA,...,45.5912,-118.73388,2.0,...,District of Oregon,1.0,1.0,1.0,1.0,1.0,1.0,umatilla_tribal_police


In [40]:
# Lists all the LEAs to check - i.e., those that don't have anything in their ORI code (not -1 --- literally nothing, showing it's uninitialized)
agency_responsible_col = "agency_responsible_1"
LEAs_to_check = df_civ.query(f"{agency_responsible_col}_ORI7.isnull() & {agency_responsible_col} != 'N/A'")
LEAs_to_check = LEAs_to_check.sort_values(by=['state_of_death', agency_responsible_col], ascending=[True, True]) # Sorts by (i) state and (ii) LEA

# Prints results, all the LEAs with an undefined ORI code
print(f"LEAs to check = {LEAs_to_check.shape[0]}")
LEAs_to_check.loc[: , [agency_responsible_col, "state_of_death"] ].head(300)

LEAs to check = 313


,agency_responsible_1,state_of_death
3110,Alabama Law Enforcement Agency,Alabama
5768,Alabama Law Enforcement Agency,Alabama
6113,Alabama Law Enforcement Agency,Alabama
707,U.S. Federal Bureau of Investigation,Alabama
4733,U.S. Marshals Service,Alabama
1143,Alaska Department of Public Safety,Alaska
330,Border Patrol,Arizona
163,Navajo Nation Police,Arizona
6370,Navajo Nation Police,Arizona
6956,Navajo Nation Police Department,Arizona


### 5.5 | Initialize Manual Corrections to Make

In [39]:
# Creates a dict to store manual corrections for agency_responsible_1, agency_responsible_2, etc.

manual_corrections_dict = {}

In [40]:
# ****************************************
# CORRECTIONS FOR AGENCY_RESPONSIBLE_1
# ****************************************

manual_corrections = {
    3110 : [420, "Alabama Law Enforcement Agency was mapped to Alabama Department of Public Safety: Former was formed in 2015, whereas latter was what came before it."],
    6113 : [420, "Alabama Law Enforcement Agency was mapped to Alabama Department of Public Safety: Former was formed in 2015, whereas latter was what came before it."],
    5768 : [420, "Alabama Law Enforcement Agency was mapped to Alabama Department of Public Safety: Former was formed in 2015, whereas latter was what came before it."],
    1143 : [592, "Alaska State Troopers were involved, a subset of the Department of Public Safety. Thus, the ORI code of the State Trooper HQ was used."],
    3215 : [820, "Pima County Probation Department doesn't have its own ORI. Thus, we usd the ORI of the HQ of Arizona's Department of Corrections, which manages probation broadly."],
    6433 : [2351, "While death occured in AZ, the sheriff appears to be from CA."],
    6656 : [1170, "Memphis, Arkansas doesn't exist. Instead, there's West Memphis, Arkansas. Thus, we used the ORI code for West Memphis Police Department."],
    4798 : [16846, "The state of death, Arkansas, does not have a Shelby County. The neighboring state, MO, does, so its ORI was taken, assuming that the Shelby sheriff's office from MO was involved."],
    2977 : [1514, "\"bastian\" county appears to be a typo for Sebastian County, Arkansas, thus Sebastian County Sheriff's ORI was used."],
    3403 : [1712, "Bay Area Rapid Transit doesn't have an ORI. It was mapped onto an agency titled 'SF Bay Rapid Transit', which appears to be the same thing."],
    6251 : [2371, "Mapped onto headquarters of California Department of Fish and Game"],
    706 : [1614, "Mapped onto Hayward Police Department. There's a typo in the name (i.e., police was spelled twice) that prevented it from automatically matching."],
    820 : [2071, "Lancaster, CA does not have its own sheriff's department. Instead, it is part of Los Angeles County, so it uses its sheriff."],	
    1398 : [2089, "LA County Probation has no ORI code. Instead, we used LA County Department of Corrections."],
    4983 : [2349, "Mapped onto Temecula PD in California."],
    5170 : [2349, "Mapped onto Temecula PD in California."],
    1752 : [2387, "Mapped onto HQ of California Park Rangers"], 
    5703 : [2147, "Mapped onto University of California, Mercedees Police "],
    433 : [3092, "There's only one transportation authority in Colorado with an ORI code. Thus, it was used."],
    5677 : [12336, "Not automatically matched because the victim perished in DC, not Maryland. This is now corrected."],
    4988 : [3782, "This is the only transit authority in Washington, DC, with an ORI code. Thus, its ORI code was used."],
    6394 : [3782, "This is the only transit authority in Washington, DC, with an ORI code. Thus, its ORI code was used."],
    7071 : [3782, "This is the only transit authority in Washington, DC, with an ORI code. Thus, its ORI code was used."],
    377 : [3783, "Mapped onto the DC Metro Police Department."],
    1498 : [3783, "Mapped onto the DC Metro Police Department."],
    4300 : [3783, "Mapped onto the DC Metro Police Department."],
    4504 : [3783, "Mapped onto the DC Metro Police Department."],
    4794 : [3783, "Mapped onto the DC Metro Police Department."],
    5010 : [3783, "Mapped onto the DC Metro Police Department."],
    5453 : [3783, "Mapped onto the DC Metro Police Department."],
    5657 : [3783, "Mapped onto the DC Metro Police Department."],
    5729 : [3783, "Mapped onto the DC Metro Police Department."],
    2875 : [3783, "Mapped onto the DC Metro Police Department."],
    537 : [3783, "Mapped onto the DC Metro Police Department."],
    5708 : [33948, "The victim perished in DC, not in VA. Thus, the program couldn't automatically link Prince George's County PD to its proper ORI. This is now fixed."],
    1422 : [5404, "Athens, GA does not have its own police department. Instead, it is merged with Clarke County. Thus, the ORI of Athens-Clarke County PD was used."],
    251 : [5661, "HQ of Georgia State Patrol was used."],
    455 : [5661, "HQ of Georgia State Patrol was used."],
    1094 : [5661, "HQ of Georgia State Patrol was used."],
    2083 : [5661, "HQ of Georgia State Patrol was used."],
    2343 : [5661, "HQ of Georgia State Patrol was used."],
    2480 : [5661, "HQ of Georgia State Patrol was used."],
    3416 : [5661, "HQ of Georgia State Patrol was used."],
    3416 : [5661, "HQ of Georgia State Patrol was used."],
    4228 : [5661, "HQ of Georgia State Patrol was used."],
    4291 : [5661, "HQ of Georgia State Patrol was used."],
    4673 : [5661, "HQ of Georgia State Patrol was used."],
    5018 : [5661, "HQ of Georgia State Patrol was used."],
    2975 : [5661, "HQ of Georgia State Patrol was used."],
    5471 : [5661, "HQ of Georgia State Patrol was used."],
    22 : [5668, "There's two ORI codes associated with the GA Department of Corrections. The investigative arm was the only one with an ORI7 and ORI9, thus it was used."],
    87 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    1095 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    1881 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    2728 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    6706 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    4360 : [3738, "Amtrak Police is an LEA with federal jurisdiction. Amtrak's HQ is in Delaware, thus, the ORI was set to its HQ."],
    5270 : [8179, "Colfax Town Marshal is apparently the name of the highest ranked police officer in Colfax, Indiana. Thus, the ORI of the police station was used."],
    3810 : [7998, "Indiana does not have a Crete Police Departments. It's neighboring state, Illinois, does; hence, the ORI from IL was used."],
    138 : [8536, "There's no ORI code associated with the Indiana State Police's emergency response department. Instead, the ORI of Indiana State Police HQ was used."],
    928 : [9334, "HQ of Iowa State Patrol was used."],
    3003 : [9334, "HQ of Iowa State Patrol was used."],
    3351 : [9334, "HQ of Iowa State Patrol was used."], 
    3019 : [10626, "There is no ORI code for West Buechel Police anywhere in the country. However, there is one code for Buechel police, and it's in Kentucky. Thus, it is assumed that the Buechel ORI also corresponds to the West Beuchel one. "],
    2711 : [11259, "The Shreveport Marshal has his own police department, which this ORI points to."],
    4345 : [12056, "Mapped to Maine State Police."],
    4351 : [12056, "Mapped to Maine State Police."],
    245 : [26467, "Victim died to Penn State Police in Maryland, hence, the program could not automatically link the ORI. This has been corrected."],
    1409 : [8246, "Victim perished in Michigan, yet Bristol PD is from Indianna, preventing program from linking ORI. This has been fixed."],
    641 : [14664, "There's only one transit police in Minnesota associated with an ORI. Thus, the Minnesota Transit Police was linked to this ORI."],
    2098 : [15273, "Mapped Memphis Police, MS to Village of Memphis Police, MS."],
    5049 : [17134, "There is no Butte PD. Instead, there's a police department that oversees the Butte-Silver Bowe area stationed in Butte. This is where the ORI was taken from."],
    2774 : [17768, "Nevada Department of Corrections does not have an ORI. However, it's headquartered in Carson City, and a Nevada Prison is headquartered there. Thus, the ORI of the latter was used."],
    3985 : [12963, "Victim perished in New Hampshire, not MA, hence, the program was not able to automatically detect MA State Police. This has been fixed."],
    2994 : [18463, "Depford Police doesn't have 'township' in its name, hence it was corrected via the proper ORI code."],
    1408 : [18742, "ORI Code Points to Howell Township Police, NJ, which is assumed to be the same as Howell Police, NJ"],
    6677 : [18742, "ORI Code Points to Howell Township Police, NJ, which is assumed to be the same as Howell Police, NJ"],
    3729 : [18588, "New Jersey Department of Corrections only appears once, in the Internal Affair Unit at its HQ in Trenton. Thus, that ORI code was used."],
    4875 : [18588, "New Jersey Department of Corrections only appears once, in the Internal Affair Unit at its HQ in Trenton. Thus, that ORI code was used."],
    6550 : [18588, "New Jersey Department of Corrections only appears once, in the Internal Affair Unit at its HQ in Trenton. Thus, that ORI code was used."],
    6861 : [18696, "Old Bridge Police, NJ doens't exist, but Old Bridge Township Police, NJ, does. Hence, the ORI code of the latter was used."],
    2884 : [18463, "Mapped onto Deptford Police. The \"Township\" was missing from the name in the CSLLEA, hence an automatic match couldn't be triggered."],
    5279 : [19086, "Mapped to LAGUNA TRIBAL POLICE department HQ, New Mexico"],
    2333 : [19487, "Fredonia Police, NY, doesn't exist, but Fredonia Village Police, NY does. Thus, the ORI code of the latter was used."],
    2334 : [19487, "Fredonia Police, NY, doesn't exist, but Fredonia Village Police, NY does. Thus, the ORI code of the latter was used."],
    5734 : [20849, "There's both a town and village of New Paltz, NY. The article revealed that the LEA involved was the town, not the village, so its ORI was used."],
    5861 : [20560, "There's no Postdam Police, NY, but there is Postdamn Village PD, NY. Thus, ORI was taken from latter."],
    2045 : [21516, "Mapped to Hertford County Sheriff."],
    6669 : [21970, "Linked to North Carolina DMV."],
    1328 : [21360, "Article confirms that agent responsible is investigator from department of motors & theft. See https://abc11.com/officer-involved-shooting-wilson-assault-robbery-officer/5697568/ "],
    5610 : [22613, "Mapped onto Cleveland Metro Parks."], 
    6960 : [34350, "Mapped to Kelso PD, which is in Washington. There is no Kelso PD in Oregon, and WA contained the closest Kelso PD, hence its ORI was taken."],
    5454 : [25686, "Mapped onto ALLEGHENY PORT AUTHORITY TRANSIT POLICE"],
    1658 : [26021, "The Attorney General has multiple narcotics investigation offices, with no clear headquarters. This death occured in Pittsburg, so we used the ORI of the office that belonged to the closest county: Butler "],
    2559 : [26682, "German Township, PA, only has one LEA with an ORI, so it was used."],
    1682 : [25623, "Penn Hills doesn't have a PD in PA, but Penn Hills Township does. Thus, the latter was used."],
    125 : [26612, "Upper Darby, NY, doesn't have a PD, but Upper Darby Township, NY, does, hence latter was used."],
    6871 : [26612, "Upper Darby, NY, doesn't have a PD, but Upper Darby Township, NY, does, hence latter was used."],
    5692 : [28785, "There are two LEAs in SC for Spartanburg College police. We chose the ORI of the methodist one."],
    571 : [9969, "There was no \"Police\" in \"Johnson City Department\", so this could not be algorithmically matched. This has been corrected."],
    5089 : [29121, "Mapped onto Pine Ridge Law Enforcement, SD."],
    7006 : [29134, "Mapped to ROSEBUD SIOUX TRIBAL POLICE, South Dakota."],
    520 : [29352, "Of all the Tennessee Highway Patrol offices, this was the only one with an ORI7 code. Hence, it was selected."],
    2103 : [29352, "Of all the Tennessee Highway Patrol offices, this was the only one with an ORI7 code. Hence, it was selected."],
    3720 : [29352, "Of all the Tennessee Highway Patrol offices, this was the only one with an ORI7 code. Hence, it was selected."],
    303 : [4509, "Victim perished in Florida, not in Tennessee as the original databases claim. After this correction, Leon County Sheriff's Office could be correctly matched."],
    3080 : [32237, "Article confirms that these officers are actually from Corpus Christi police department: https://www.kristv.com/news/2018/04/16/two-corpus-christi-police-officers-placed-back-on-duty/"],
    5634 : [19140, "Jal Police is in New Mexico, not Texas. The ORI code fixes this"],
    5376 : [31311, "There's multiple Harris County Constables. This ensures the correct precinct - 6 - is linked to this LEA."],
    2431 : [30100, "There is no San Antonio Park Police. There is, however, San Atonio Park Rangers, so the ORI of that LEA was used."],
    3016 : [30100, "There is no San Antonio Park Police. There is, however, San Atonio Park Rangers, so the ORI of that LEA was used."],
    6971 : [30100, "There is no San Antonio Park Police. There is, however, San Atonio Park Rangers, so the ORI of that LEA was used."],
    3255 : [3296, "Although death was in Utah, Sheriff is stationed in Colorado. This ORI fixes that."],
    6751 : [3296, "Although death was in Utah, Sheriff is stationed in Colorado. This ORI fixes that."],
    1044 : [12470, "Death occured in VA, yet death was inflicted by Montgomery County Police in MD. This ORI corrects this."],
    6865 : [6508, "Death occure din WA, yet Lewiston PD is in Idaho. Hence, the Idaho ORI was used"],
    1798 : [34731, "Mapped to fish & wildlife enforcement"],
}

# Adds to Manual Corrections Dict
manual_corrections_dict["agency_responsible_1_corrections"] = manual_corrections

In [41]:
# ****************************************
# CORRECTIONS FOR AGENCY_RESPONSIBLE_2
# ****************************************

manual_corrections = {
    5451 : [2366, "Mapped to RANCHO CORDOV PD, California"],
    1207 : [8739, "The Lynn Town Marshal appears to be a police officer, and his office is in the Lynn Town Police Department. Thus, they're assumed to be the same thing, and the ORI was therefore mapped to the Lynn Town Police. See https://www.in.gov/towns/lynn/our-staff/"],
    6303 : [9946, "Mapped to capitol police (Kansas / Topeka)"],
    902 : [17021, "Mapped to Blackfeet LES, Montana"],
    2834 : [25426, "Mapped to UMATILLA TRIBAL POLICE DEPARTMENT, Oregon"],
    169 : [17649, "Las Vegas State Polie to Nevada State Police (highway patrol), the Las Vegas division"],
    173 : [17649, "Las Vegas State Polie to Nevada State Police (highway patrol), the Las Vegas division"],
    531 : [15445 , "Brookhaven Police. Even though state of death was in Georgia, assumed the MS Brookhaven Police got involved"],
    673 : [19807, "Le Roy Police to Le Roy Village Police"],
    798 : [18415, "Fairfield Police to Fairfield Township Police"],
    976 : [420, "Alabama Law Enforcement Agency was mapped to Alabama Department of Public Safety: Former was formed in 2015, whereas latter was what came before it."],
    1205 : [33330, "Utah Highway Patorl to Headquarters; we used the headquarters rather than any satellite LEAs"],
    1969 : [33330, "Utah Highway Patorl to Headquarters; we used the headquarters rather than any satellite LEAs"],
    4285 : [33330, "Utah Highway Patorl to Headquarters; we used the headquarters rather than any satellite LEAs"],
    6595 : [33330,"Utah Highway Patorl to Headquarters; we used the headquarters rather than any satellite LEAs"],
    59 : [33330,"Utah Highway Patorl to Headquarters; we used the headquarters rather than any satellite LEAs"],
    1974 : [5661, "Georgia State patrol mapped onto Headquarters in Atlanta"],
    2343 : [5382, "Savannah PD merged with Chatham to form Savannah-Chatham PD. Thus, we mapped Savannah PD onto Savanah-Chatham"],
    2740 : [29325, "Mapped Highway Patrol to Nashville; Nashville office is its headquarters"],
    2821 : [2031, "Death occured in Pomona, yet no CA State Police is in Pomona. Instead, we used California Polytech State Police, which is in Pomona."],
    4907 : [13678, "Only one branch of MI department of corrections has an ORI: Lansing. Thus, we used that."],
    5057 : [12680, "No Fall River PD in state of death (RI). Instead, there was one in MA, so we used that."],
    5577 : [23184, "Oregon police ot Oregon police; nothing to note"],
    6032 : [5349, "Caroll County is in Georgia, so the ORI was mapped appropriately"],
    6454 : [32934, "TX department of corrections is headquartered in Huntsville, so that location was used"],
    7097 : [32806, "Texas Rangers are headquartered in Austin, so that location was used "],
    7101 : [7159, "Wasketa Police doesn't exist, but Watseka does. Thus, it was assumed to be a typo"],
    1976 : [13678,  "Only one branch of MI department of corrections has an ORI: Lansing. Thus, we used that"],
    4907 : [13678,  "Only one branch of MI department of corrections has an ORI: Lansing. Thus, we used that"],
}

# Adds to Manual Corrections Dict
manual_corrections_dict["agency_responsible_2_corrections"] = manual_corrections

In [42]:
# ****************************************
# CORRECTIONS FOR AGENCY_RESPONSIBLE_3
# ****************************************

manual_corrections = {
    4447 : [5661, "HQ of Georgia State Patrol was used."],
    798 : [18903, "We mapped Wayne Police to Wayne Township Police because the former does not exist."], 
    1142 : [17765, "Mapped Nevada Highway Patrol onto its headquarters."],
    2406 : [2371,  "CA Fish & Wildlife. Multiple offices, so we selected the generic one that wasn't specific to a region."],
    6712 : [13678,  "Only one branch of MI department of corrections has an ORI: Lansing. Thus, we used that"],
}

# Adds to Manual Corrections Dict
manual_corrections_dict["agency_responsible_3_corrections"] = manual_corrections

AGENCY_RESPONSIBLE_4 and AGENCY_RESPONSIBLE_5 need no manual corrections

### 5.6 | Apply Manual Corrections

In [43]:
# Statistics
total_lea_counter, missing_ori_counter, corrected_ori_counter = 0, 0, 0

# Loops through each agency_responsible column (i.e., #1-#5)
for agency_responsible_num in range(1,6):

    # Inits key vars
    agency_responsible_col = f"agency_responsible_{agency_responsible_num}"
    agency_correction_key = f"{agency_responsible_col}_corrections"
    print(f"\n{'*'*80}\nAPPLYING MANUAL CORRECTIONS TO [{agency_responsible_col}]\n{'*'*80}\n")

    # Stats: Missing values
    total_LEAs = df_civ.query(f"{agency_responsible_col} != 'N/A'").shape[0]
    total_lea_counter += total_LEAs
    missed_LEAs = df_civ.query(f"{agency_responsible_col}_ORI7.isnull() & {agency_responsible_col} != 'N/A'") 
    missing_ori_counter += missed_LEAs.shape[0]    
    print(f"Of the {total_LEAs} agencies in [{agency_responsible_col}], {missed_LEAs.shape[0]} have missing values ({ round( missed_LEAs.shape[0] / total_LEAs * 100, 2)})%")

    # If there's no manual corrections to make, continue with the loop
    if agency_correction_key not in manual_corrections_dict.keys():
        print("No manual matches found.")
        continue
        
    # Stats: Manual Corrections
    manual_corrections = manual_corrections_dict[agency_correction_key]
    corrected_LEAs = len(manual_corrections.keys())
    corrected_ori_counter += corrected_LEAs
    print(f"Applying {corrected_LEAs} manual corrections to the {missed_LEAs.shape[0]} agencies with missing ORIs, thereby remedying {round( corrected_LEAs / missed_LEAs.shape[0] * 100, 2)}% of the data")

    # Goes through each LEA with a missing value (either due to N/A or due to a genuine missing value)
    for index, ind_LEA in missed_LEAs.iterrows():
    
        # Applies manual correctioon, if it exists
        if index in manual_corrections.keys():
            ori_index = manual_corrections[index][0]
            match_to_add = initialize_match_to_add_for_dmfs_civs( df_ori.loc[ ori_index ], agency_responsible_col) 

            # Records manual match for transparency & replication
            record_manual_match("dmfs_civilians", agency_responsible_num, ind_LEA[agency_responsible_col], df_ori.loc[ori_index, "LEA_NAME"].strip().title(), 
                                ori_index, df_ori.loc[ori_index, "ORI7"], df_ori.loc[ori_index, "ORI9"], manual_corrections[index][1])

        # Else, just create a bunch of blank/NA cells to add
        else:
            match_to_add = initialize_match_to_add_for_dmfs_civs( pd.Series(), agency_responsible_col )
            record_manual_match("dmfs_civilians", agency_responsible_num, ind_LEA[agency_responsible_col], "No Match Found", -1, -1, -1, notes="")
        
        # Adds data to DM-FS
        df_civ.loc[index, match_to_add.keys()] =  match_to_add.values()


********************************************************************************
APPLYING MANUAL CORRECTIONS TO [agency_responsible_1]
********************************************************************************

Of the 7105 agencies in [agency_responsible_1], 313 have missing values (4.41)%
Applying 116 manual corrections to the 313 agencies with missing ORIs, thereby remedying 37.06% of the data

********************************************************************************
APPLYING MANUAL CORRECTIONS TO [agency_responsible_2]
********************************************************************************

Of the 467 agencies in [agency_responsible_2], 64 have missing values (13.7)%
Applying 28 manual corrections to the 64 agencies with missing ORIs, thereby remedying 43.75% of the data

********************************************************************************
APPLYING MANUAL CORRECTIONS TO [agency_responsible_3]
********************************************************

In [44]:
# Replaces any remaining missing values relates to ORIs.
# These values only exists if there was no agency_responsible_2, agency_responsible_3, or agency_responsible_4, as there were no agencies by which ORIs & its associated data could be pulled from

print("There are still 'None' values in agency_responsible if agency_responsible was N/A. For example, the ORI code for agency_responsible_3 will be None if there weren't 3 agencies responsible for someone's death")
print("Let's fill these missing values with appropriate indicators.")

for agency_responsible_num in range(1,6):
    agency_responsible_col = f"agency_responsible_{agency_responsible_num}"
    print(f"Applying missing value correction to {agency_responsible_col}")

    # Fill with -1
    df_civ[f"{agency_responsible_col}_ORI7"] = df_civ[f"{agency_responsible_col}_ORI7"].fillna(-1)
    df_civ[f"{agency_responsible_col}_ORI9"] = df_civ[f"{agency_responsible_col}_ORI9"].fillna(-1)
    df_civ[f"{agency_responsible_col}_FIPS_STATE"] = df_civ[f"{agency_responsible_col}_FIPS_STATE"].fillna(-1)
    df_civ[f"{agency_responsible_col}_FIPS_COUNTY"] = df_civ[f"{agency_responsible_col}_FIPS_COUNTY"].fillna(-1)

    # Fill with NA
    df_civ[f"{agency_responsible_col}_LEA_TYPE"] = df_civ[f"{agency_responsible_col}_LEA_TYPE"].fillna("N/A")
    df_civ[f"{agency_responsible_col}_LEA_SUBTYPE_1"] = df_civ[f"{agency_responsible_col}_LEA_SUBTYPE_1"].fillna("N/A")
    df_civ[f"{agency_responsible_col}_LEA_SUBTYPE_2"] = df_civ[f"{agency_responsible_col}_LEA_SUBTYPE_2"].fillna("N/A")
    df_civ[f"{agency_responsible_col}_LOCAL_GOVERNMENT_NAME"] = df_civ[f"{agency_responsible_col}_LOCAL_GOVERNMENT_NAME"].fillna("N/A")
    df_civ[f"{agency_responsible_col}_DISTRICT"] = df_civ[f"{agency_responsible_col}_DISTRICT"].fillna("N/A")

There are still 'None' values in agency_responsible if agency_responsible was N/A. For example, the ORI code for agency_responsible_3 will be None if there weren't 3 agencies responsible for someone's death
Let's fill these missing values with appropriate indicators.
Applying missing value correction to agency_responsible_1
Applying missing value correction to agency_responsible_2
Applying missing value correction to agency_responsible_3
Applying missing value correction to agency_responsible_4
Applying missing value correction to agency_responsible_5


In [45]:
# Prints total statistics
print(f"Total LEAs = {total_lea_counter} | LEAs with missing ORIs = {missing_ori_counter} ({round(missing_ori_counter/total_lea_counter * 100, 2)}% of total) "
      f"| Corrected LEAs with missing values = {corrected_ori_counter} ({ round( corrected_ori_counter / missing_ori_counter * 100, 2)})% of missing ORIs")

Total LEAs = 7688 | LEAs with missing ORIs = 391 (5.09% of total) | Corrected LEAs with missing values = 149 (38.11)% of missing ORIs


### 5.7 | Exports

In [46]:
# Exports DMFS-CIV via EXPORT_DIR
df_civ.drop(columns=["agency_responsible_1_CLEANED", "agency_responsible_2_CLEANED", "agency_responsible_3_CLEANED", "agency_responsible_4_CLEANED", "agency_responsible_5_CLEANED"], inplace=True)
df_civ.to_csv(f"{EXPORT_DIR}DMFS Civilians.csv", index=False)

In [47]:
# Also exports any manual corrections we've made
print(f"Exporting manual corrections to [{MATCH_DIR}]")
manual_corrections_df.to_csv(f"{MATCH_DIR}manual_ori_corrections.csv", index = False)

Exporting manual corrections to [res/3.0 - Manual ORI Matches/]


In [48]:
# Exports Manual Match Dataframe, the one that records all our manual matches (if appliable)

if not USE_PRE_EXISTING_MATCH_DATAFRAME:
    print(f"Saving manual ORI matches to dir [{MATCH_DIR}] so that the manual matches can be replicated in a future run.")
    match_verification_df.to_csv(f"{MATCH_DIR}manual_ori_matches.csv", index = False)
else:
    print("Used pre-existing manual match dataframe. That dataframe was not modified in any way, hence, no saving is necessary.\nNo action is needed. Please proceed.")

Used pre-existing manual match dataframe. That dataframe was not modified in any way, hence, no saving is necessary.
No action is needed. Please proceed.
